# Data Cleaning Notebook

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [9]:
# read the dataset using the compression zip
df_006 = pd.read_csv('data/corona_tested_individuals_ver_006.english.csv')
df_0083 = pd.read_csv('data/corona_tested_individuals_ver_0083.english.csv')

C:\Users\joshp\AppData\Local\Temp\ipykernel_28904\1587404722.py:2: DtypeWarning: Columns (1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_006 = pd.read_csv('data/corona_tested_individuals_ver_006.english.csv')
C:\Users\joshp\AppData\Local\Temp\ipykernel_28904\1587404722.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_0083 = pd.read_csv('data/corona_tested_individuals_ver_0083.english.csv')


In [6]:
df_006.dtypes

test_date              object
cough                  object
fever                  object
sore_throat            object
shortness_of_breath    object
head_ache              object
corona_result          object
age_60_and_above       object
gender                 object
test_indication        object
dtype: object

In [22]:
def convertStrToDate(df,columns):
    """
    Converts columns from the list of provided column names 
    into datetime64[ns] dtype.
    
    Returns: pd.DataFrame
    
    """
    _df = df.copy()
    
    for col in columns:
        _df[col] = pd.to_datetime(_df[col])
        
    return _df
    

In [17]:
df = convertStrToDate(df_006,["test_date"])

## Renaming features

In [29]:
df['covid_result'] = df['corona_result'].map({
    'negative':0,
    'positive':1,
    'other':-1
})

In [32]:
df['close_contact'] = df['test_indication'].map(lambda x: 1 if x == 'Contact with confirmed' else 0)

In [36]:
def convertColumnsToInt(df,columns):
    _df = df.copy()
    for col in columns:
        _df[col] = _df[col].astype('int64')
        
    return _df

In [35]:
df[df["close_contact"]==1]

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication,covid_result,close_contact
9,2020-04-30,0,0,0,0,0,negative,None,male,Contact with confirmed,0,1
23,2020-04-30,1,0,0,0,0,negative,None,male,Contact with confirmed,0,1
41,2020-04-30,1,0,0,0,0,positive,None,male,Contact with confirmed,1,1
103,2020-04-30,0,0,0,0,0,positive,None,female,Contact with confirmed,1,1
120,2020-04-30,0,0,0,0,0,positive,None,male,Contact with confirmed,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
278827,2020-03-11,0,0,0,0,0,negative,None,None,Contact with confirmed,0,1
278828,2020-03-11,0,0,0,0,0,negative,None,None,Contact with confirmed,0,1
278830,2020-03-11,0,1,0,0,0,positive,None,None,Contact with confirmed,1,1
278836,2020-03-11,0,0,0,0,0,positive,None,None,Contact with confirmed,1,1


In [10]:
num_columns = ["cough","fever","sore_throat","shortness_of_breath","head_ache"]
cat_columns = ["age_60_and_above","gender","test_indication"]

In [16]:
for col in num_columns:
    df_006[col] = pd.to_numeric(df_006[col],errors='coerce')
df_006.dropna(inplace=True)
df_006[num_columns] = df_006[num_columns].astype("int64")

In [30]:
df = pd.get_dummies(df_006,columns=cat_columns)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278594 entries, 0 to 278847
Data columns (total 16 columns):
 #   Column                                  Non-Null Count   Dtype         
---  ------                                  --------------   -----         
 0   test_date                               278594 non-null  datetime64[ns]
 1   cough                                   278594 non-null  int64         
 2   fever                                   278594 non-null  int64         
 3   sore_throat                             278594 non-null  int64         
 4   shortness_of_breath                     278594 non-null  int64         
 5   head_ache                               278594 non-null  int64         
 6   corona_result                           278594 non-null  object        
 7   age_60_and_above_No                     278594 non-null  uint8         
 8   age_60_and_above_None                   278594 non-null  uint8         
 9   age_60_and_above_Yes                 

In [24]:
ohe = OneHotEncoder()

In [34]:
df = df[df['gender_None'] != 1]
df["gender_None"].value_counts()

0    259042
Name: gender_None, dtype: int64